In [5]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tyaju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\tyaju\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tyaju\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tyaju\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [7]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [8]:
words = []
classes = []
documents = []
ignore = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

In [10]:
classes = sorted(list(set(classes)))
print(len(documents),"documents")
print(len(classes),"classes",classes)
print(len(words),"unique stemmed words",words)

31 documents
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 unique stemmed words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [23]:
words

["'s",
 'acceiv',
 'address',
 'anyon',
 'ar',
 'bye',
 'can',
 'card',
 'cash',
 'cours',
 'credit',
 'day',
 'del',
 'delicy',
 'delivery',
 'dish',
 'do',
 'food',
 'for',
 'from',
 'good',
 'goodby',
 'hello',
 'help',
 'hi',
 'hom',
 'hour',
 'how',
 'is',
 'lat',
 'loc',
 'main',
 'mastercard',
 'me',
 'menu',
 'most',
 'on',
 'op',
 'opt',
 'provid',
 'resta',
 'see',
 'serv',
 'situ',
 'spec',
 'tak',
 'tel',
 'thank',
 'that',
 'the',
 'ther',
 'today',
 'what',
 'when',
 'wher',
 'yo',
 'you']

In [12]:
training = []
output = []
output_empty = [0]*len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])

In [13]:
random.shuffle(training)
training = np.array(training)

In [14]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [15]:
tf.reset_default_graph()

In [16]:
net = tflearn.input_data(shape=[None,len(train_x[0])])
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net,tensorboard_dir='tflearn_logs')

model.fit(train_x,train_y,n_epoch=1000,batch_size=8,show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.38638 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.38638 - acc: 0.9686 -- iter: 24/31
Training Step: 4000  | total loss: 0.35166 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.35166 - acc: 0.9717 -- iter: 31/31
--
INFO:tensorflow:D:\JUPYTER NOTEBOOK\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [17]:
import pickle
pickle.dump({'words':words,'classes':classes,'train_x':train_x,'train_y':train_y},open('training_data.pkl','wb'))

In [19]:
data = pickle.load(open('training_data.pkl' , 'rb'))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [20]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [21]:
model.load('./model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from D:\JUPYTER NOTEBOOK\model.tflearn


In [22]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence , words , show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if(w==s):
                bag[i]=1
                if show_details:
                    print("found in bag: " , w)
                    
    return(np.array(bag))


In [28]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    results = model.predict([bow(sentence,words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x:x[1] ,  reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list
    
def response(sentence, userID='123' , show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))
                
            results.pop(0)
    

In [31]:
classify('what are your woking hours ?')

[('hours', 0.94408107)]

In [32]:
response('what are your working hours ?')

Our hours are 9am-9pm every day


In [41]:
response('that is helpful!')

My pleasure


In [39]:
response('bye !')

Bye! Come back again soon.


In [42]:
#adding the context

In [43]:
context = {}
ERROR_THRESHOLD = 0.25
def classify(sentence):
    results = model.predict([bow(sentence,words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]],r[1]))
    return return_list

def response(sentence, userID='123' , show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        if show_details: print('context:' , i[context_details])
                        context[userID] = i['context_set']
                        
                    if not 'context_filter' in i or (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        return print(random.choice(i['responses']))
                        
            results.pop(0)
                        

In [46]:
response('what delivery options you have?')

We have home delivery options through UBER Eats and Zomato
